In [ ]:
# !pip install earthstat

In [ ]:
from earthstat.earthstat import EarthStat

In [ ]:
# predictor_name              = 'FPAR'
# predictor_dir               = 'FPAR_500'
# mask_path                   = 'crop mask/asap_mask_crop_v04.tif'
# shapefile_path              = 'shapefile/gaul1_asap.shp'
# selected_countries          = ["Norway","Spain"] #
# country_column_name         = 'adm0_name'		   # Name of Column contains countries
# invalid_values              = [255, 254, 251]

In [ ]:
# aggregate_fpar = EarthStat(predictor_name)

In [ ]:
# aggregate_fpar.initDataDir(predictor_dir)

Data Summary:
- Found 7 TIFF files in 'FPAR_500'.
- Data covers the period from 2015-07-11 to 2015-10-21.
- Predictor: FPAR
- Resolution: 80640x29346
- Coordinate Reference System (CRS): WGS 84


Predictor Initialized Correctly, Initialize The Mask


In [ ]:
# aggregate_fpar.initMask(mask_path)

Mask Initialized Correctly, Initialize The Shapefile


In [ ]:
# aggregate_fpar.initShapefile(shapefile_path)

Shapefile Initialized Correctly, Now Check The Data Compatibility


In [ ]:
# aggregate_fpar.DataCompatibility()

NO ISSUE: The pixel sizes of the mask and predictor are identical: (0.004464285715, 0.004464285715)
NO ISSUE: The projections of the mask and predictor are identical: WGS 84
NO ISSUE: The projections of the raster data and shapefile are identical: WGS 84

COMPATIBILITY CHECK PASSED: The data is compatible. No resolution or projection mismatches were detected.


In [ ]:
# aggregate_fpar.fixCompatibilityIssues()

Checking for compatibility issues...
No compatibility issues detected. Predictor, mask, and shapefile are already compatible.


In [ ]:
# aggregate_fpar.selectRegionOfInterest(selected_countries,
#                                       country_column_name)

Filtered shapefile saved to: shapefile/filtered_gaul1_asap.shp
Region of Interest (ROI) successfully selected based on the specified countries: Viet Nam, Spain.


In [ ]:
# aggregate_fpar.clipPredictor(invalid_values)

Clipping Rasters: 100%|██████████| 7/7 [01:41<00:00, 14.49s/it]

Clipping operation successful. Clipped data stored in: clipped FPAR_500


In [ ]:
# aggregate_fpar.runAggregation()

Starting aggregation with the selected Region of Interest (ROI) for FPAR.


Processing rasters:   0%|          | 0/7 [00:00<?, ?raster/s]

Aggregation complete. Data saved to Aggregated_FPAR.csv.
